<a href="https://colab.research.google.com/github/podo47/Apriori-Algorithm/blob/main/Apriori_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Apriori Algorithm

## Import library

In [1]:
from itertools import combinations
import csv
from collections import Counter
import pandas as pd

## Mount google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Calculate support

In [3]:
def supports(item,trans):
  num_trans = len(trans)
  # Count the frequency of each item
  item_freq = dict()
  for i in item:
    if type(i) is tuple:
      set_i = set(i)
    else:
      set_i = {i}
    for j in range(num_trans):
      if set_i.issubset(set(trans[j])):
        if i in item_freq:
          item_freq[i] += 1
        else:
          item_freq[i] = 1
  return item_freq

## Calculate confidence

In [4]:
def confidence(candidate,min_conf):
  len_can = len(candidate)
  conf_list = []
  for i in range(len_can-1):
    conf_dict = dict()
    for j in candidate[i].items():
      if type(j[0]) is tuple:
        left = set(j[0])
      else:
        left = {j[0]}
      for k in candidate[i+1].items():
        right = set(k[0])
        if left.issubset(right):
          conf = k[1]/j[1]
          left_i = (j[0],k[0])
          if conf>min_conf:
            conf_dict[left_i] = conf
    conf_list.append(conf_dict)

  return conf_list

## Answer form

In [5]:
def ans(conf_list):
  conf_len = len(conf_list)
  with open('/content/drive/MyDrive/Apriori_Algorithm/output.csv', 'w', newline='') as csvfile:
    fieldnames = ['Rule:left','→','Rule:right','Confidence']
    csvfile = csv.writer(csvfile)
    csvfile.writerow(fieldnames)
    for i in range(conf_len):
      for j in conf_list[i].items():
        if type(j[0][0]) is tuple: 
          left = set(j[0][0]) 
        else:
          left = {j[0][0]}
          left_single = list(map(str, left))
        right = set(j[0][1]) - left
        right = list(map(str, right))
        if len(left)==1:
          csvfile.writerow([left_single[0],'→',right[0],j[1]])
        else:
          csvfile.writerow([left,'→',right[0],j[1]])


## Main : Apriori algorithm

In [6]:
def Apriori():
  min_supp = float(input("Please enter min support : ")) # User-defined : 0 < input < 1 
  min_conf = float(input("Please enter min confidence : ")) # User-defined : 0 < input <= 1 

  item = set()
  candidate = list()
  count = []
  for i in trans:
    for j in i:
      count.append(j)
  count = Counter(count)
  supp_sum = sum(count.values())
  min_supp = min_supp*supp_sum
  candidate.append({j[0]:j[1] for j in count.items() if j[1]>min_supp})
  for i in count.items():
    if i[1]>min_supp:
      item.add(i[0])
  max_trans_size = len(item)
  n = 2
  i = 1
  while n < max_trans_size:
    if len(item)<n:
      break
    item_combin = combinations(item,n)
    supp_combin = supports(item_combin,trans)
    if list({j[0]:j[1] for j in supp_combin.items() if j[1]>min_supp}) != []:
        candidate.append({j[0]:j[1] for j in supp_combin.items() if j[1]>min_supp})
    if len(candidate)-1 < i:
      break
    item_test = set()
    for j in candidate[i].keys():
      for k in j:
        item_test.add(k)
    item = item_test
    n+=1
    i+=1

  strong_rule = confidence(candidate,min_conf)
  answer = ans(strong_rule)

  return answer

# Run

## Input file preprocessing

### Read input

In [7]:
path = '/content/drive/MyDrive/Apriori_Algorithm/input.txt'

### Modify input format and store in list

In [8]:
trans = []
with open(path) as ip:
  for line in ip:
      sp = line.split()
      sp = [int(i) for i in sp]
      trans.append(sp)

## Output

In [9]:
test = Apriori() #Please enter min support : 0.001/ Please enter min confidence : 0.05
df = pd.read_csv ('/content/drive/MyDrive/Apriori_Algorithm/output.csv')
df

Please enter min support : 0.001
Please enter min confidence : 0.05


,Rule:left,→,Rule:right,Confidence
0,32,→,38,0.186787
1,32,→,39,0.557460
2,32,→,41,0.210721
3,32,→,48,0.529703
4,36,→,38,0.950272
...,...,...,...,...
163,"{48, 110, 38}",→,39,0.757531
164,"{48, 41, 39}",→,32,0.223459
165,"{48, 41, 39}",→,38,0.270296
166,"{48, 170, 39}",→,38,0.989221
